In [37]:
import sklearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()  # for plot styling

from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D # for 3D plots

In [38]:
data = pd.read_csv('./Data/OutputDatos2.csv')
data.head()

,titleType,startYear,runtimeMinutes,numVotes,region,main_genre,secondary_genre
0,movie,1991,137,1128166,US,Action,Sci-Fi
1,tvSeries,2000,22,577,US,Action,Adventure
2,movie,2002,150,860569,US,Biography,Drama
3,movie,2004,120,417391,US,Action,Crime
4,movie,2005,132,5946,US,Biography,Drama


In [39]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7462 entries, 0 to 7461
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   titleType        7462 non-null   object
 1   startYear        7462 non-null   int64 
 2   runtimeMinutes   7462 non-null   int64 
 3   numVotes         7462 non-null   int64 
 4   region           7462 non-null   object
 5   main_genre       7462 non-null   object
 6   secondary_genre  7462 non-null   object
dtypes: int64(3), object(4)
memory usage: 408.2+ KB


In [40]:
data.describe()

,startYear,runtimeMinutes,numVotes
count,7462.000000,7462.000000,7.462000e+03
mean,2013.162557,100.792147,7.288656e+04
std,6.977416,64.615235,1.641005e+05
min,1990.000000,3.000000,1.000000e+01
25%,2008.000000,60.000000,6.265250e+03
50%,2015.000000,101.000000,1.527900e+04
75%,2019.000000,122.000000,5.648475e+04
max,2023.000000,990.000000,2.197234e+06


In [42]:
data[data_objects].describe()

,titleType,region,main_genre,secondary_genre
count,7462,7462,7462,7462
unique,4,4,22,26
top,movie,IN,Drama,Drama
freq,4676,5590,1767,2426


In [43]:
for o  in data_objects:
    le = LabelEncoder()
    le.fit(data[o])
    data[o] = le.transform(data[o])

In [46]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7462 entries, 0 to 7461
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   titleType        7462 non-null   int32
 1   startYear        7462 non-null   int64
 2   runtimeMinutes   7462 non-null   int64
 3   numVotes         7462 non-null   int64
 4   region           7462 non-null   int32
 5   main_genre       7462 non-null   int32
 6   secondary_genre  7462 non-null   int32
dtypes: int32(4), int64(3)
memory usage: 291.6 KB


In [44]:
data.head()

,titleType,startYear,runtimeMinutes,numVotes,region,main_genre,secondary_genre
0,0,1991,137,1128166,3,0,19
1,3,2000,22,577,3,0,0
2,0,2002,150,860569,3,3,6
3,0,2004,120,417391,3,0,4
4,0,2005,132,5946,3,3,6
